Popuate random data and export as CSV

In [34]:
import csv
import random
import datetime
from datetime import timedelta
from faker import Faker

fake = Faker()

usual_pattern = [
    (datetime.time(7, 0), datetime.time(17, 0), False),  # 7am - 5pm: Light Off
    (datetime.time(17, 0), datetime.time(22, 0), True),  # 5pm - 10pm: Light On (sometimes Off)
    (datetime.time(22, 0), datetime.time(2, 0), False),  # 10pm - 2am: Light Off (sometimes On)
    (datetime.time(2, 0), datetime.time(2, 15), True),  # 2am - 2:15am: Light On (mostly Off)
    (datetime.time(2, 15), datetime.time(7, 0), False)   # 2:15am - 7am: Light Off (mostly On)
]

with open("home_assistant_log.csv", "w", newline='') as file:
    writer = csv.writer(file)
    #header = ['hour_of_day', 'day_of_week', 'month', 'lightOn', 'timestamp']  # Define the header
    header = ["lightOn","timestamp"]
    writer.writerow(header)

    for i in range(0, 1000):
        random_datetime = fake.date_time_between(start_date='-30d', end_date='now')

        light_on = False

        for start_time, end_time, light_status in usual_pattern:
            if start_time < random_datetime.time() < end_time:
                #if(random.randint(1,10)%2):
                #    light_on = True
                writer.writerow([light_status, random_datetime])
                break

        # Randomly generate hour of the day, day of the week, and month
        #hour_of_day = random.randint(0, 23)
        #day_of_week = random.randint(0, 6)
        #month = random.randint(1, 12)

        #random_timestamp = (start_date + (end_date - start_date) * random.random()).timestamp() # Generate UNIX timestamp
        #hour = random_timestamp.hour
        #print(hour)

        #timestamp = int((start_date + (end_date - start_date) * random.random()).timestamp())  # Generate UNIX timestamp

        # Write the row to the CSV file
        #writer.writerow([hour_of_day, day_of_week, month, lightOn, timestamp])


Machine Learning

In [33]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import calendar

def checkActualOutputOfLight(date):
    for start_time, end_time, light_status in usual_pattern:
        if start_time < date.time() < end_time:
            return "ON" if light_status else "OFF"

# Read the CSV file
df = pd.read_csv("home_assistant_log.csv")

# Define features (X) and target (y)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df['hour_of_day'] = df['timestamp'].dt.hour
df["minute"] = df["timestamp"].dt.minute

X = df.drop(columns=["timestamp", "lightOn"], axis=1).to_numpy()
y = df["lightOn"].to_numpy()

#print(X)

## Initialize and train the model
model = DecisionTreeClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print("Accuracy is {}".format(accuracy_score(y_test, y_pred)))

ranDt = [fake.date_time() for _ in range(10)]
list_of_days = list(calendar.day_abbr)

for i in ranDt:
    rand_date = pd.to_datetime(i)
    hour = rand_date.hour
    day = rand_date.day_of_week
    #month = rand_date.month
    minute = rand_date.minute
    #prediction = model.predict([[hour,day,minute]])
    prediction = model.predict([ [hour,minute]])


    #print("At {},{}:{}, light should be {}".format(list_of_days[day-1], hour, minute, "on" if prediction else "off"))
    print("At {:02d}:{:02d}, light predicted to {} -- Actual: {}".format( hour, minute, "on" if prediction else "off", checkActualOutputOfLight(rand_date) ))


Accuracy is 0.96875
At 19:22, light predicted to on -- Actual: ON
At 07:21, light predicted to off -- Actual: OFF
At 19:40, light predicted to on -- Actual: ON
At 14:35, light predicted to off -- Actual: OFF
At 15:34, light predicted to off -- Actual: OFF
At 09:28, light predicted to off -- Actual: OFF
At 15:39, light predicted to off -- Actual: OFF
At 20:17, light predicted to on -- Actual: ON
At 07:13, light predicted to off -- Actual: OFF
At 20:41, light predicted to on -- Actual: ON
